In [1]:
%load_ext memory_profiler

## Run benchmark with

From repo root/base folder

```bash
mprof run python extract_point_from_raster_buffer.py -f srg-dev/test-data/pop_density/pop_density/*.tif -g srg-dev/test-data/1000_testing_points.rds
```

## Run all the cells below to record peak memory and time

In [2]:
import numpy as np
import pandas as pd
# import geopandas as gpd
import rioxarray as riox
import dask.dataframe as dd

from pyproj import Transformer
from shapely.geometry import mapping, Point

start = pd.Timestamp('now')
buffer_value = 10_000

## Run the analysis in a scalable way

load the raster and fill NaNs with 0

In [3]:
myraster = (
    riox.open_rasterio('test-data/pop_density/pop_density/apg18e_1_0_0_20210512.tif')
    .sel(band=1)
)

In [4]:
myraster = myraster.where(myraster != myraster.rio.nodata, drop=True)

In [5]:
"{:,}".format(myraster.data.shape[0] * myraster.data.shape[1])

'15,138,272'

Load points for data extraction and create buffers

In [6]:
transformer = Transformer.from_crs("EPSG:3577", myraster.rio.crs, always_xy=True)

In [7]:
points = (
    pd.read_csv('test-data/1000_testing_points.csv')
    .rename(columns={'X': 'x', 'Y': 'y'})
    .assign(
        lat_lon_tuple = lambda columns: columns[['x', 'y']].apply(lambda row: transformer.transform(row['x'], row['y']), axis=1),
        lat = lambda columns: columns['lat_lon_tuple'].apply(lambda el: el[0]),
        lon = lambda columns: columns['lat_lon_tuple'].apply(lambda el: el[1]),
        points = lambda columns: columns['lat_lon_tuple'].apply(Point),
        points_buffer = lambda columns: columns['points'].apply(lambda x: x.buffer(buffer_value))
    )
)
points = dd.from_pandas(points, npartitions=24)# assign partitions equal to 2 x Nr logical cores in my machine

Memory of dataframe in MB

In [8]:
# def extract_mean_from_buffer(raster, geom):
#     data_points = pd.Series(geom.exterior.coords)
#     values_from_raster = data_points.apply(lambda row: raster.sel(x=row[0], y=row[1], method="nearest").item()).values
#     return values_from_raster.mean()
def extract_mean_from_buffer(raster, geom):
    data_points = geom.exterior.coords
    raster_selection = raster.sel(
        x=[el[0] for el in data_points],
        y=[el[1] for el in data_points],
        method="nearest"
    )
    return np.diag(raster_selection.data).mean()

In [9]:
points['extracted_mean'] = points['points_buffer'].apply(
    lambda x: extract_mean_from_buffer(myraster, x),
    meta=float
) * myraster.attrs['scale_factor'] + myraster.attrs['add_offset']

In [10]:
points = points.compute(scheduler="processes")

In [11]:
print(f"running time: {pd.Timestamp('now') - start}")

running time: 0 days 00:00:06.397729


In [12]:
points.sample(20)

,x,y,lat_lon_tuple,lat,lon,points,points_buffer,extracted_mean
894,929688.143328,-3.184429e+06,"(929688.1433277295, -3184428.5805817773)",929688.143328,-3.184429e+06,POINT (929688.1433277295 -3184428.580581777),POLYGON ((939688.1433277295 -3184428.580581777...,0.000000
670,904228.185962,-3.184429e+06,"(904228.185961943, -3184428.5805817773)",904228.185962,-3.184429e+06,POINT (904228.185961943 -3184428.580581777),"POLYGON ((914228.185961943 -3184428.580581777,...",0.000000
70,836031.871589,-3.184429e+06,"(836031.8715893005, -3184428.5805817773)",836031.871589,-3.184429e+06,POINT (836031.8715893005 -3184428.580581777),POLYGON ((846031.8715893005 -3184428.580581777...,0.000000
83,837509.458401,-3.184429e+06,"(837509.4584007077, -3184428.5805817773)",837509.458401,-3.184429e+06,POINT (837509.4584007077 -3184428.580581777),POLYGON ((847509.4584007077 -3184428.580581777...,0.000000
540,889452.317848,-3.184429e+06,"(889452.3178478705, -3184428.5805817773)",889452.317848,-3.184429e+06,POINT (889452.3178478705 -3184428.580581777),POLYGON ((899452.3178478705 -3184428.580581777...,0.017554
737,911843.441067,-3.184429e+06,"(911843.4410668882, -3184428.5805817773)",911843.441067,-3.184429e+06,POINT (911843.4410668882 -3184428.580581777),POLYGON ((921843.4410668882 -3184428.580581777...,0.000000
241,855467.821186,-3.184429e+06,"(855467.8211855036, -3184428.5805817773)",855467.821186,-3.184429e+06,POINT (855467.8211855036 -3184428.580581777),POLYGON ((865467.8211855036 -3184428.580581777...,0.000000
19,830235.184868,-3.184429e+06,"(830235.1848676258, -3184428.5805817773)",830235.184868,-3.184429e+06,POINT (830235.1848676258 -3184428.580581777),POLYGON ((840235.1848676258 -3184428.580581777...,0.000000
616,898090.517668,-3.184429e+06,"(898090.5176684052, -3184428.5805817773)",898090.517668,-3.184429e+06,POINT (898090.5176684052 -3184428.580581777),POLYGON ((908090.5176684052 -3184428.580581777...,0.000000
957,936848.756337,-3.184429e+06,"(936848.7563368572, -3184428.5805817773)",936848.756337,-3.184429e+06,POINT (936848.7563368572 -3184428.580581777),POLYGON ((946848.7563368572 -3184428.580581777...,0.000000
